# train 특수기호 전처리

- 처음엔 apply 로 빠르게 처리하고 넘어갔었는데 이렇게 하면 start_index와 end_index가 바뀐다는 것을 고려하지 못했었습니다..
- 그래서 apply 적용 후, .index() 메소드를 사용하려 했지만.. 해당 entity가 문장에 2번 나오는 경우도 있어서
- 어쩔수없이 하나하나 전처리로 고쳐나갔습니다.

In [57]:
import pandas as pd
train = pd.read_csv("../input/data/train/train.tsv",sep='\t',header=None)
test = pd.read_csv("../input/data/test/test.tsv",sep='\t',header=None)


In [58]:
def num_punc(data,pattern):
  return sum(data[1].apply(lambda x: True if pattern in x else False))
punc_list = ["『","』","=","+","#","/","\\","?",":",";","^","$",".","@","*","\"","※","~","&","%","·","!","|","(",")","{","}","[","]","<",">","`","\'","《","》","…"]

train_num_punc_dict = {punc:num_punc(train,punc) for punc in punc_list}
test_num_punc_dict = {punc:num_punc(test,punc) for punc in punc_list}

tr_no_punc=[punc for punc in punc_list if num_punc(train,punc)==0]
tr_exist_punc=[punc for punc in punc_list if num_punc(train,punc)!=0]

te_no_punc=[punc for punc in punc_list if num_punc(test,punc)==0]
te_exist_punc=[punc for punc in punc_list if num_punc(test,punc)!=0]

In [59]:
# for k,v in train_num_punc_dict.items():
#   print(f"{k} : {v}개")

print(f"train과 test 모두 없는 기호들 : {list(set(tr_no_punc)&set(te_no_punc))}")
print(f"train에만 있고 test에 없는 기호들 : {list(set(tr_exist_punc)-set(te_exist_punc))}")
print(f"test에만 있고 train에 없는 기호들 : {list(set(te_exist_punc)-set(tr_exist_punc))}")
print(f"train과 test 모두 있는 기호들 : {list(set(tr_exist_punc)&set(te_exist_punc))}")

print(f"외부데이터 추가시에 포함되어선 안될 특수기호들 : {list(set(punc_list)-set(te_exist_punc))}")

train과 test 모두 없는 기호들 : ['{', '<', '>', '\\', '+', '}', '?', '^', '#', '…', '※', '@']
train에만 있고 test에 없는 기호들 : ['`', '『', '*', '$', '=', '』']
test에만 있고 train에 없는 기호들 : []
train과 test 모두 있는 기호들 : [']', '.', '[', '》', "'", '&', '~', '"', '(', '!', ';', ')', '/', ':', '《', '|', '·', '%']
외부데이터 추가시에 포함되어선 안될 특수기호들 : ['{', '<', '>', '`', '『', '\\', '*', '+', '$', '?', '}', '^', '#', '…', '=', '※', '@', '』']


In [60]:
new_train = train.copy()

## 1. * 은 (4개) 
replace('*','')로 단순 제거해주면 됨

In [61]:
new_train.iloc[3825,:]
new_train.iloc[3825,1]

'2012년 10월 18일에는 B*Witched가 ITV2의 텔레비전 리얼리티 다큐멘터리 시리즈 《더 빅 리유니언》(The Big Reunion)에 출연하기 위하여 당대의 다른 5개의 팝 음악 그룹인 허니즈(Honeyz), 911, 리버티 X(Liberty X), 5ive, 아토믹 키튼(Atomic Kitten)와 함께 재결합할 것이라는 보도가 전해졌다.'

In [62]:
new_train.iloc[3825,1] = new_train.iloc[3825,1].replace('*','')
new_train.iloc[3825,2] = new_train.iloc[3825,2].replace('*','')
new_train.iloc[3825,[4,6,7]] -= 1

In [63]:
new_train.iloc[2304,1] = new_train.iloc[2304,1].replace('*','')
# new_train.iloc[2304,5] = new_train.iloc[2304,5].replace('*','')
new_train.iloc[2304,[3,4,6,7]] -= 4

In [64]:
new_train.iloc[2243,1] = new_train.iloc[2243,1].replace('*','')
# new_train.iloc[2243,5] = new_train.iloc[2243,5].replace('*','')
# new_train.iloc[2243,7] = 46

In [65]:
new_train.iloc[1624,1] = new_train.iloc[1624,1].replace('*','')
new_train.iloc[1624,5] = new_train.iloc[1624,5].replace('*','')
new_train.iloc[1624,7] = 46

In [66]:
# new_train[1] = new_train[1].apply(lambda x: x.replace('*',''))

## 2. = (데이터 2개)

replace('本方=일본','일본') 이건 놔두기

replace('시마즈=아리마','시마즈-아리마')


In [67]:
new_train[1] = new_train[1].apply(lambda x: x.replace('시마즈=아리마','시마즈-아리마'))

In [68]:
new_train.iloc[7213].values

array(['wikipedia-2934-12-36-43-10-15',
       '1584년 일어난 오키타나와테 전투에서 당시 류조지 가문의 당주 류조지 다카노부가 시마즈-아리마 연합군에 패사하였다.',
       '류조지 다카노부', 36, 43, '오키타나와테', 10, 15, '인물:사망_원인'], dtype=object)

## 3. 『랑  』(쌍으로 8개)
『 는 replace('『','《') 이중꺾쇠로 바꾸자

』 는 replace('』','》') 이중꺾쇠로 바꾸자

In [69]:
new_train[1] = new_train[1].apply(lambda x: x.replace('『','《').replace('』','》'))

## 4. $ (5개)

R$ 는 replace('R$','금액') 으로 대체하자.

AU$ 는 replace('AU$','금액') 으로 대체하자.

$ 는 replace('$','금액') 으로 대체하자.


In [70]:
new_train.loc[new_train[1].apply(lambda x: True if '$' in x else False)].index

Int64Index([401, 525, 1412, 5913, 7795], dtype='int64')

In [71]:
new_train.iloc[7795].values

array(['wikipedia-2209-59-5-6-20-22',
       '위원회는 정부의 공식적인 사과와 각 생존자에게 $20,000의 보상을 지불하는 등의 법적인 구제를 권장하였다.',
       '정부', 5, 6, '생존자', 20, 22, '관계_없음'], dtype=object)

In [73]:
new_train.iloc[7795,1] = new_train.iloc[7795,1].replace('$','금액')
new_train.iloc[7795,1]

'위원회는 정부의 공식적인 사과와 각 생존자에게 금액20,000의 보상을 지불하는 등의 법적인 구제를 권장하였다.'

In [74]:
new_train.iloc[5913,1] = new_train.iloc[5913,1].replace('$','금액')
new_train.iloc[5913,1]

'그는 Ludacris(루다크리스)와 폰티액 간에 금액1,200만 캠페인 거래를 중개하여 자신의 마케팅 비즈니스를 시작하였고, Pontiac의 광고에는 노래가 나오는 한편, Ludacris의 1시간에 2마일" 뮤직 비디오에는 Pontiac이 나오게 된다.'

In [76]:
new_train.iloc[1412,1] = new_train.iloc[1412,1].replace('R$','금액')
new_train.iloc[1412,1]

'필리포 인차기 신임 감독의 계획에서 제외되었음에도 불구하고, 이탈리아 구단 측이 호비뉴의 주급을 지불하고 "물고기 군단"측이 월급으로 금액400,000을 부담하고, 밀란이 금액400,000을 책임질 것으로 발표되었다.'

In [77]:
new_train.iloc[525,1] = new_train.iloc[525,1].replace('AU$','금액')
new_train.iloc[525,[3,4,6,7]] -= 1
new_train.iloc[525,1]

'2011년 7월 비도시치는 금액320,000의 이적료로 A리그 애들레이드 유나이티드 FC와 3년 계약을 맺으며, 오스트레일리아로 복귀했다.'

In [78]:
new_train.iloc[401,1] = new_train.iloc[401,1].replace('$','금액')
new_train.iloc[401,1]

'이지연 신영증권 연구원은 “SK이노베이션의 올해 1분기 영업이익은 -1180억원으로 적자가 불가피할 전망”이라며 “코로나19의 영향으로 운송수요가 급감하였으며, 평년보다 따뜻한 날씨로 정제마진은 지난해 4분기 금액2.1에서 올해 1분기 금액0.5로 하락한 상황”이라고 말했다.'

In [468]:
# new_train[1] = new_train[1].apply(lambda x: x.replace('R$','금액').replace('AU$','금액').replace('$','금액'))

## 5. ` (7개)

replace("괴물 신인`","괴물 신인")

나머진 replace("`","'") 로 대체하자

In [34]:
new_train.loc[new_train[1].apply(lambda x: True if '`' in x else False)].index

Int64Index([2482, 5071, 6075, 6237, 6560, 6801, 8482], dtype='int64')

In [79]:
new_train.iloc[5071,1] = new_train.iloc[5071,1].replace("괴물 신인`","괴물 신인")
new_train.iloc[5071,[3,4,6,7]] -= 1
new_train.iloc[5071,1]

"괴물 신인 ITZY(있지)가 JYP엔터테인먼트(이하 JYP) 박진영 대표가 작사, 작곡한 `ICY(아이씨)'로 컴백한다."

In [80]:
new_train[1] = new_train[1].apply(lambda x: x.replace("`","'"))

##  Augmentation
Minority Class에 대해서만 적용하였습니다.

### 1. 대체어로 replace 채워넣기
- 동의어 혹은 유사어를 통해 채워넣는 방법도 생각해봤지만, 인물이라는 특성상 이는 합리적이지 않다고 결론내림
- 이 방법은 train의 다른 라벨을 가지는 데이터에서 단어를 가져오는 것이기 때문에,
- 학습시 해당 entity에 대한 noise로 작용하여 일반화 성능을 높일 수 있는 좋은 방법일 것이라고 생각함

### 2. Pororo 번역
- 번역으로(중국어, 일본어를 경유하는) augmentation
- 번역을 거치며 해당 데이터에 Noise가 생성되고 train valid split을 통해 Noise를 모델이 Denosing하기를 기대해볼 수 있다.
- 하지만 그 의미가 유사하며, entity의 변화도 없기 때문에 Data Leakage Issue가 있다고 볼 수도 있다.
- 현재 Task에서는 Minority Class에 대해 따로 Penalty가 적용되는 metric이 아니어서 이렇다 하더라도 별 위험은 없다.
- 모든 라벨을 고르게 맞추기 위한 목적의 프로젝트였다면?


In [82]:
import re
person_nick = new_train.loc[new_train.iloc[:,8]=='인물:별칭',2]
# 한글로 쓰여진 인물 이름 중에 2명 무작위로 고름
person_nick = person_nick.loc[person_nick.apply(lambda x: True if not re.match(r'[^가-힣]',x) else False)]
import random
random.seed(1234)
idx1,idx2,idx3,idx4 = random.sample(range(len(person_nick)),4)

person1 = person_nick.values[idx1]
person2 = person_nick.values[idx2]
person3 = person_nick.values[idx3]
person4 = person_nick.values[idx4]
print(person1)
print(person2)
print(person3)
print(person4)


레오나르도
나빌 페키르
루다크리스
강지환


In [83]:
def add_data_replace_person(data,label):
  the_label_data = data.loc[data.iloc[:,8]==label,:]
  new_sentence1 = the_label_data.apply(lambda x: x.iloc[1].replace(x.iloc[2],person1),axis=1)
  new_data1 = the_label_data.copy()
  new_data1.iloc[:,1] = new_sentence1
  new_data1.iloc[:,2] = person1
  if the_label_data.shape[0]>=3:
    return new_data1
  new_sentence2 = the_label_data.apply(lambda x: x.iloc[1].replace(x.iloc[2],person2),axis=1)
  new_data2 = the_label_data.copy()
  new_data2.iloc[:,1] = new_sentence2
  new_data2.iloc[:,2] = person2
  if the_label_data.shape[0]>=2:
    return pd.concat([new_data1,new_data2],axis=0)
  new_sentence3 = the_label_data.apply(lambda x: x.iloc[1].replace(x.iloc[2],person3),axis=1)
  new_data3 = the_label_data.copy()
  new_data3.iloc[:,1] = new_sentence3
  new_data3.iloc[:,2] = person3
  new_sentence4 = the_label_data.apply(lambda x: x.iloc[1].replace(x.iloc[2],person4),axis=1)
  new_data4 = the_label_data.copy()
  new_data4.iloc[:,1] = new_sentence4
  new_data4.iloc[:,2] = person4
  return pd.concat([new_data1,new_data2,new_data3,new_data4],axis=0)

In [84]:
label = '인물:사망_국가'
aug_replace_die_country = add_data_replace_person(train,label)
label = '인물:사망_도시'
aug_replace_die_city = add_data_replace_person(train,label)
label = '인물:출생_도시'
aug_replace_born_city = add_data_replace_person(train,label)
new_train = pd.concat([new_train,aug_replace_die_country,aug_replace_die_city,aug_replace_born_city],axis=0)
new_train.index = range(new_train.shape[0])
# new_train

In [85]:
new_train.shape

(9011, 9)

In [86]:
new_train.loc[new_train.iloc[:,8]=='인물:사망_국가']

,0,1,2,3,4,5,6,7,8
6631,wikipedia-14464-241-6-12-15-18,7월 1일 다리우스 3세가 박트리아에서 베수스에게 암살되었다.,다리우스 3세,6,12,박트리아,15,18,인물:사망_국가
9000,wikipedia-14464-241-6-12-15-18,7월 1일 레오나르도가 박트리아에서 베수스에게 암살되었다.,레오나르도,6,12,박트리아,15,18,인물:사망_국가
9001,wikipedia-14464-241-6-12-15-18,7월 1일 나빌 페키르가 박트리아에서 베수스에게 암살되었다.,나빌 페키르,6,12,박트리아,15,18,인물:사망_국가
9002,wikipedia-14464-241-6-12-15-18,7월 1일 루다크리스가 박트리아에서 베수스에게 암살되었다.,루다크리스,6,12,박트리아,15,18,인물:사망_국가
9003,wikipedia-14464-241-6-12-15-18,7월 1일 강지환가 박트리아에서 베수스에게 암살되었다.,강지환,6,12,박트리아,15,18,인물:사망_국가


In [90]:
from pororo import pororo, Pororo
mt = Pororo(task="mt", lang="multi", model="transformer.large.multi.mtpg")
mt

[TASK]: MT
[LANG]: MULTI
[MODEL]: transformer.large.multi.mtpg

In [87]:
import numpy as np
import re
def pororo_augmentation(origin_kor_series, lang):
  mask1 = 'QWER'
  mask2 = 'ZXCV'
  origin_kor = origin_kor_series.iloc[1]
  origin_kor = origin_kor.replace(origin_kor_series.iloc[2],'QWER').replace(origin_kor_series.iloc[5],'ZXCV')
  translated = mt.predict(origin_kor,src='ko',tgt=lang)
  new_kor = mt.predict(translated,src=lang,tgt='ko')
  # print(origin_kor)
  # print(new_kor)
  pat1 = re.compile(r'[qQ][wW][eE][rR]')
  pat2 = re.compile(r'[zZ][xX][cC][vV]')
  pat3 = re.compile(r'[가-힣]')
  new_kor_series = origin_kor_series.copy()
  # if mask1 in new_kor and mask2 in new_kor:
  if re.search(pat1,new_kor) and re.search(pat2,new_kor) and re.search(pat3,new_kor):
    # new_kor_series.iloc[1] = new_kor.replace('QWER',origin_kor_series.iloc[2]).replace('ZXCV',origin_kor_series.iloc[5])
    recover_mask2 = re.sub(r'[zZ][xX][cC][vV]',origin_kor_series.iloc[5],new_kor)
    new_kor_series.iloc[1] = re.sub(r'[qQ][wW][eE][rR]',origin_kor_series.iloc[2],recover_mask2)
    return new_kor_series
  else:
    new_kor_series.iloc[1] = np.nan
    return new_kor_series

def pororo_per_label_from_data(data,label = '인물:출생_도시'):
  origin_kor_df = data.loc[data.iloc[:,8]==label,:]
  ja_df = origin_kor_df.apply(lambda x: pororo_augmentation(x, 'ja'),axis=1)
  if origin_kor_df.shape[0]>=7:
    return ja_df
  zh_df = origin_kor_df.apply(lambda x: pororo_augmentation(x, 'zh'),axis=1)
  return pd.concat([ja_df,zh_df],axis=0)

In [88]:
new_train.iloc[:,8].value_counts()

관계_없음          4441
단체:구성원          815
인물:소속단체         679
인물:직업/직함        553
단체:상위_단체        335
단체:별칭           231
인물:출신성분/국적      209
인물:동료           186
단체:제작           165
인물:부모님          164
단체:본사_도시        136
단체:본사_국가        115
인물:별칭           103
인물:배우자           92
인물:자녀            84
단체:하위_단체         80
단체:창립일           67
인물:기타_친족         58
인물:제작            56
인물:형제/자매/남매      52
단체:창립자           45
인물:사망_일시         45
단체:모회사           44
인물:출생_일시         36
인물:거주_국가         27
인물:거주_도시         26
단체:본사_주(도)       23
단체:정치/종교성향       18
단체:해산일           15
인물:종교            15
인물:거주_주(도)       12
인물:용의자           11
단체:주주            11
인물:학교             9
단체:구성원_수          9
인물:출생_도시          8
인물:출생_국가          8
인물:사망_원인          7
인물:사망_도시          6
단체:자회사            5
인물:나이             5
인물:사망_국가          5
Name: 8, dtype: int64

In [91]:
label = '인물:나이'
aug_person_age = pororo_per_label_from_data(new_train,label)
label = '단체:자회사'
aug_sub_company = pororo_per_label_from_data(new_train,label)
label = '인물:출생_도시'
aug_born_city = pororo_per_label_from_data(new_train,label)
label = '인물:사망_국가'
aug_die_country = pororo_per_label_from_data(new_train,label)
label = '인물:사망_도시'
aug_die_city = pororo_per_label_from_data(new_train,label)
label = '인물:학교'
aug_die_school = pororo_per_label_from_data(new_train,label)
label = '인물:사망_원인'
aug_die_cause = pororo_per_label_from_data(new_train,label)
label = '인물:출생_국가'
aug_born_country = pororo_per_label_from_data(new_train,label)
label = '단체:구성원_수'
aug_num_org = pororo_per_label_from_data(new_train,label)
label = '단체:주주'
aug_holder = pororo_per_label_from_data(new_train,label)
label = '인물:용의자'
aug_suspect = pororo_per_label_from_data(new_train,label)

new_train = pd.concat([new_train,aug_person_age,aug_sub_company,aug_born_city,aug_die_country,aug_die_city,
                      aug_die_cause,aug_born_country,aug_num_org,aug_holder,aug_suspect,aug_die_school],axis=0)
new_train.index = range(new_train.shape[0])


In [92]:
new_train.shape

(9116, 9)

In [93]:
new_train.shape

(9116, 9)

In [94]:
print(new_train.shape)
# print(new_train.dropna().shape)
new_train_drop = new_train.dropna()
print(new_train_drop.shape)
# new_train.drop_duplicates(inplace=True)
new_train_drop = new_train_drop.drop_duplicates()
print(new_train_drop.drop_duplicates().shape)


(9116, 9)
(9108, 9)
(9108, 9)


In [95]:
new_train_drop.iloc[:,8].value_counts()

관계_없음          4441
단체:구성원          815
인물:소속단체         679
인물:직업/직함        553
단체:상위_단체        335
단체:별칭           231
인물:출신성분/국적      209
인물:동료           186
단체:제작           165
인물:부모님          164
단체:본사_도시        136
단체:본사_국가        115
인물:별칭           103
인물:배우자           92
인물:자녀            84
단체:하위_단체         80
단체:창립일           67
인물:기타_친족         58
인물:제작            56
인물:형제/자매/남매      52
단체:창립자           45
인물:사망_일시         45
단체:모회사           44
인물:출생_일시         36
인물:거주_국가         27
인물:거주_도시         26
단체:본사_주(도)       23
인물:용의자           22
단체:주주            21
단체:정치/종교성향       18
단체:구성원_수         18
인물:학교            18
인물:출생_도시         16
인물:출생_국가         16
인물:종교            15
인물:사망_국가         15
단체:해산일           15
인물:사망_도시         14
인물:사망_원인         14
인물:나이            14
단체:자회사           13
인물:거주_주(도)       12
Name: 8, dtype: int64

In [96]:
sum(new_train_drop.iloc[:,1].apply(lambda x: x.startswith(' ')))

1

In [97]:
sum(test.iloc[:,1].apply(lambda x: x.startswith(' ')))

0

# 규진님 데이터 적정 비율 추가

In [101]:
god_kyujin_data.iloc[0,1]

'카터 는 이집트 와 이스라엘 을 조정 하 여 , 캠프 데이비드 에서 안와르 사다트 대통령 과 메나헴 베긴 수상 과 함께 중동 평화 를 위한 캠프데이비드 협정 을 체결 했 다 . '

In [100]:
god_kyujin_data.iloc[0,1].index('안와르 사다트')

38

In [99]:
god_kyujin_data = pd.read_csv("../input/data/process_external_data/all_csv.tsv",sep='\t',header=None)
god_kyujin_data

,0,1,2,3,4,5,6,7,8
0,from train_csv 5,"카터 는 이집트 와 이스라엘 을 조정 하 여 , 캠프 데이비드 에서 안와르 사다트 ...",안와르 사다트,38,45,이집트,5,8,인물:출신성분/국적
1,from train_csv 8,"카터 는 이집트 와 이스라엘 을 조정 하 여 , 캠프 데이비드 에서 안와르 사다트 ...",캠프데이비드 협정,78,87,이스라엘,11,15,단체:구성원
2,from train_csv 9,"카터 는 이집트 와 이스라엘 을 조정 하 여 , 캠프 데이비드 에서 안와르 사다트 ...",캠프데이비드 협정,78,87,이집트,5,8,단체:구성원
3,from train_csv 11,"카터 는 이집트 와 이스라엘 을 조정 하 여 , 캠프 데이비드 에서 안와르 사다트 ...",메나헴 베긴,52,58,이스라엘,11,15,인물:출신성분/국적
4,from train_csv 65,선거 전 까지 각축전 을 벌인 유력 한 후보 는 15 대 대통령 선거 에 나와 김대...,한나라당,63,67,이회창,75,78,단체:구성원
...,...,...,...,...,...,...,...,...,...
229606,from dev_csv 99215,"2. ""사슴여인"" (박건호 작사 / 계동균 작곡) – 03:24",박건호,11,14,사슴여인,4,8,인물:제작
229607,from dev_csv 99216,"9. ""안녕히 계세요"" (장덕 작사 / 장덕 작곡) - 02:06",장덕,14,16,안녕히 계세요,4,11,인물:제작
229608,from dev_csv 99217,"2009년, 블런트는 장마크 발레가 감독을 맡고 줄리언 펠로스가 각본을 쓴 《영 빅...",줄리언 펠로스,27,34,영 빅토리아,43,49,인물:제작
229609,from dev_csv 99218,2008년 저드 애퍼토가 제작하고 세스 로건과 에번 골드버그가 각본을 맡은 코미디 ...,에번 골드버그,26,33,파인애플 익스프레스,50,60,인물:제작


In [103]:
# god_kyujin_data = pd.read_csv("../input/data/process_external_data/all_csv.tsv",sep='\t',header=None)
god_kyujin_data.iloc[:,1] = god_kyujin_data.iloc[:,1].apply(lambda x: x.strip())
from transformers import AutoTokenizer
model_name_from_pretrained = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name_from_pretrained)
process_vc = god_kyujin_data.iloc[:,8].value_counts()

# tokenizer 적용후, train에서의 token len보다 길이가 긴 데이터는 제거
new_train_drop['token_len'] = new_train_drop.iloc[:,1].apply(lambda x: len(tokenizer(x)['input_ids']))
test['token_len'] = test.iloc[:,1].apply(lambda x: len(tokenizer(x)['input_ids']))
god_kyujin_data['token_len'] = god_kyujin_data.iloc[:,1].apply(lambda x: len(tokenizer(x)['input_ids']))

def min_max_len_filtering(data):
  condition = (new_train_drop['token_len'].min()<data['token_len'])&(data['token_len']<new_train_drop['token_len'].max())
  return data.loc[condition,:]

print(new_train_drop['token_len'].min(),new_train_drop['token_len'].max())
print(test['token_len'].min(),test['token_len'].max())
print(god_kyujin_data['token_len'].min(),god_kyujin_data['token_len'].max())
print(god_kyujin_data.shape)

god_kyujin_data = min_max_len_filtering(god_kyujin_data)
god_kyujin_data.shape

Token indices sequence length is longer than the specified maximum sequence length for this model (675 > 512). Running this sequence through the model will result in indexing errors


13 341
13 224
5 5615
(229611, 10)


(225578, 10)

In [162]:
god_kyujin_data.iloc[:,8].value_counts()

인물:출신성분/국적    77353
인물:직업/직함      34449
단체:창립일        26586
인물:제작         16311
단체:제작         13662
단체:구성원        13643
인물:배우자        10986
인물:자녀         10398
인물:기타_친족       9619
인물:부모님         4546
단체:상위_단체       3436
인물:학교          2045
단체:정치/종교성향     1545
인물:소속단체         875
단체:창립자          124
Name: 8, dtype: int64

In [163]:
# print(f"train과 test 모두 없는 기호들 : {list(set(tr_no_punc)&set(te_no_punc))}")
print(f"train에만 있는 기호들 : {list(set(tr_exist_punc)-set(te_exist_punc))}")
print(f"test에만 있는 기호들 : {list(set(te_exist_punc)-set(tr_exist_punc))}")
print(f"train과 test 모두 있는 기호들 : {list(set(tr_exist_punc)&set(te_exist_punc))}")

print(f"외부데이터 추가시에 포함되어선 안될 특수기호들 : {list(set(punc_list)-set(te_exist_punc))}")

train에만 있는 기호들 : ['`', '『', '*', '$', '=', '』']
test에만 있는 기호들 : []
train과 test 모두 있는 기호들 : [']', '.', '[', '》', "'", '&', '~', '"', '(', '!', ';', ')', '/', ':', '《', '|', '·', '%']
외부데이터 추가시에 포함되어선 안될 특수기호들 : ['{', '<', '>', '`', '『', '\\', '*', '+', '$', '?', '}', '^', '#', '…', '=', '※', '@', '』']


In [164]:
import re
def exist_punc(word):
  reg = re.compile('[?><+*※^…=『』`@$#\{\}\\\]')
  if reg.search(word):
    # print("특수문자 포함")
    # print(reg.search(word))
    return True
  else:
    # print("특수문자 없음")
    # print(reg.search(word))
    return False

word = 'asdf\\'
print(word)
exist_punc(word)

asdf\


True

In [165]:
print(god_kyujin_data.shape)
god_kyujin_data_filtered = god_kyujin_data.loc[god_kyujin_data.iloc[:,1].apply(lambda x: not exist_punc(x)),:]
print(god_kyujin_data_filtered.shape)

(225578, 10)
(219878, 10)


In [166]:
god_kyujin_data_filtered.iloc[:,8].value_counts()

인물:출신성분/국적    75758
인물:직업/직함      34053
단체:창립일        26010
인물:제작         15658
단체:제작         13315
단체:구성원        13156
인물:배우자        10373
인물:자녀          9980
인물:기타_친족       9327
인물:부모님         4346
단체:상위_단체       3376
인물:학교          2019
단체:정치/종교성향     1512
인물:소속단체         871
단체:창립자          124
Name: 8, dtype: int64

In [167]:
new_train_drop

,0,1,2,3,4,5,6,7,8,token_len,ner_e1,ner_e2
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작,50,"[(랜드로버, ORGANIZATION)]","[(자동차, ARTIFACT)]"
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음,41,"[(민주당, ORGANIZATION)]","[(27석, QUANTITY)]"
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭,52,"[(유럽 축구 연맹, ORGANIZATION)]","[(UEFA, ORGANIZATION)]"
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함,101,"[(강수일, PERSON)]","[(공격수, CIVILIZATION)]"
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님,44,"[(람캄, LOCATION), (행, O)]","[(퍼쿤, PERSON), ( , O), (씨, O), ( , O), (인트라팃, O)]"
...,...,...,...,...,...,...,...,...,...,...,...,...
9103,wikitree-55169-1-15-17-23-27,10일 매체는 방송인 서장훈이 모교 연세대학교에 광고 모델료 1억5000만원을 전액...,서장훈,12,14,연세대학교,23,27,인물:학교,33,"[(서장훈, PERSON)]","[(연세대학교, ORGANIZATION)]"
9104,wikipedia-29295-6-74-76-60-64,"이때 리천희는 주포 양희승이 아킬레스건 상처를 입었을 때, 주전 선수들이 고려대학교...",이규섭,74,76,고려대학교,60,64,인물:학교,103,"[(이규섭, PERSON)]","[(고려대학교, ORGANIZATION)]"
9105,wikipedia-9102-0-0-5-49-53,다쓰노 긴고(1854년 10월 13일~1919년 3월 25일)은 일본의 건축가이다....,다쓰노 긴고,0,5,고부대학교,49,53,인물:학교,59,"[(다쓰노 긴고, PERSON)]","[(고부대학교, ORGANIZATION)]"
9106,wikipedia-14613-6-76-81-22-27,"1949년~1950년에는 교환학생으로 파리 대학교에 머물렀고, 여기서 장 룰레 안리...",자크 디미에,76,81,파리 대학교,22,27,인물:학교,54,"[(자크 디미에, PERSON)]","[(파리 대학교, ORGANIZATION)]"


In [168]:
random.seed(77)

process_vc = god_kyujin_data_filtered.iloc[:,8].value_counts()
new_train_vc = new_train_drop.iloc[:,8].value_counts()
min_freq = process_vc.iloc[-1] # 124

vc_train_intersection_process = new_train_vc[process_vc.index].sort_values(ascending=False)
min_freq_name = process_vc.index[process_vc.argmin()]

num_chosen_dict = {}
min_freq = process_vc[min_freq_name]
num_chosen_dict[min_freq_name] = min_freq

threshold = vc_train_intersection_process[min_freq_name]
log_p = 2
for freq_name in [pv_id for pv_id in process_vc.index if pv_id!=min_freq_name]:
  # min_freq/current_freq, 데이터 존재비율 반영하여 라벨별 데이터 수 다르게 추가
  count_freq_per_min = threshold/vc_train_intersection_process[freq_name]
  ## 로그함수 이용 : 데이터를 추가함에 있어 라벨 별 비율을 반영할 때 단순 선형적 비율로 판단하면 안된다는 생각.
  if log_p != 0:
    transformed_sample_prop = np.log(count_freq_per_min+(log_p-1))/np.log(log_p)  # 0보다 큰 값이 나와야 함.
  num_chosen_dict[freq_name] = min(int(np.ceil(transformed_sample_prop * min_freq)), process_vc[freq_name])
print(f'num_chosen_dict : {num_chosen_dict}')
chosen_index = []
for process_unique_label in set(god_kyujin_data_filtered.iloc[:,8]):
  chosen_index.extend(random.sample(god_kyujin_data_filtered.loc[god_kyujin_data_filtered.iloc[:,8]==process_unique_label].index.tolist(),num_chosen_dict[process_unique_label]))
process_external_data = god_kyujin_data_filtered.loc[chosen_index]

num_chosen_dict : {'단체:창립자': 124, '인물:출신성분/국적': 35, '인물:직업/직함': 14, '단체:창립일': 92, '인물:제작': 106, '단체:제작': 44, '단체:구성원': 10, '인물:배우자': 72, '인물:자녀': 77, '인물:기타_친족': 103, '인물:부모님': 44, '단체:상위_단체': 23, '인물:학교': 225, '단체:정치/종교성향': 225, '인물:소속단체': 12}


In [169]:
# 최종 정제된 규진님의 외부 데이터
process_external_data

,0,1,2,3,4,5,6,7,8,token_len
228557,from dev_csv 5135,2014년 MBC 주말특별기획 《마마 (드라마)》 ... 박남순 역,MBC,6,9,마마 (드라마),18,26,단체:제작,20
200054,from train_csv 283731,2014 년 tvN 월화 드라마 《 라이어 게임 》,tvN,7,10,라이어 게임,20,26,단체:제작,16
203340,from train_csv 350859,《 막돼먹은 영애씨 》 는 tvN 에서 방송 되 고 있 는 다큐멘터리 텔레비전 프로...,tvN,15,18,막돼먹은 영애씨,2,10,단체:제작,42
197460,from train_csv 242038,정재형 이효리의 유&아이 ( 2012 . 02 . 26 ~ 2012 . 10 . 1...,SBS TV,51,57,정재형 이효리의 유&아이,1,14,단체:제작,35
199427,from train_csv 268241,"KBS 뉴스라인 ( KBS 제1TV , 밤 11 시 )",KBS 제1TV,12,20,KBS 뉴스라인,1,9,단체:제작,16
...,...,...,...,...,...,...,...,...,...,...
229050,from dev_csv 8366,이에 활동이 멈출 위기에 놓였으나 휴렛 팩커드의 창업자인 윌리엄 휴렛과 데이브 팩커...,폴 앨런,65,69,마이크로소프트,49,56,단체:창립자,81
222113,from train_csv 975423,"1832년, 제임스 매터슨과 청나라 광저우(沙面島)에서 ‘자딘 매터슨’ 상회를 설립했다.",제임스 매터슨,8,15,자딘 매터슨,33,39,단체:창립자,36
229030,from dev_csv 8346,"유한양행과 라초이를 설립한 기업가 유일한의 동생이자, 유유제약을 설립한 유특한의 형이다.",유특한,40,43,유유제약,30,34,단체:창립자,36
229005,from dev_csv 8321,현재 그는 T.I.가 설립한 레이블인 그랜드 허슬 레코드에 소속되어 있다.,T.I.,6,10,그랜드 허슬 레코드,21,31,단체:창립자,25


In [170]:
process_external_data.index = range(process_external_data.shape[0])

# Pororo 이용 ner Tag 붙이기
- new_train
- process_external_data

In [110]:
from pororo import Pororo
ner = Pororo(task="ner", lang="ko")

In [111]:
ner("이순신은 조선 중기의 무신이다")

[('이순신', 'PERSON'),
 ('은', 'O'),
 (' ', 'O'),
 ('조선 중기', 'DATE'),
 ('의', 'O'),
 (' ', 'O'),
 ('무신', 'CIVILIZATION'),
 ('이다', 'O')]

In [112]:
ner('유리가가린')

[('유리', 'MATERIAL'), ('가가린', 'O')]

In [113]:
new_train_drop

,0,1,2,3,4,5,6,7,8,token_len
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작,50
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음,41
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭,52
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함,101
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님,44
...,...,...,...,...,...,...,...,...,...,...
9111,wikitree-55169-1-15-17-23-27,10일 매체는 방송인 서장훈이 모교 연세대학교에 광고 모델료 1억5000만원을 전액...,서장훈,15,17,연세대학교,23,27,인물:학교,33
9112,wikipedia-29295-6-74-76-60-64,"이때 리천희는 주포 양희승이 아킬레스건 상처를 입었을 때, 주전 선수들이 고려대학교...",이규섭,74,76,고려대학교,60,64,인물:학교,103
9113,wikipedia-9102-0-0-5-49-53,다쓰노 긴고(1854년 10월 13일~1919년 3월 25일)은 일본의 건축가이다....,다쓰노 긴고,0,5,고부대학교,49,53,인물:학교,59
9114,wikipedia-14613-6-76-81-22-27,"1949년~1950년에는 교환학생으로 파리 대학교에 머물렀고, 여기서 장 룰레 안리...",자크 디미에,76,81,파리 대학교,22,27,인물:학교,54


In [114]:
new_train_drop['ner_e1'] = new_train_drop.iloc[:,2].apply(lambda x: ner(x))
new_train_drop['ner_e2'] = new_train_drop.iloc[:,5].apply(lambda x: ner(x))

In [158]:
new_train_drop.index = range(new_train_drop.shape[0])

In [175]:
ner_train = new_train_drop.copy()
ner_train.to_csv('../input/data/train/ner_train.tsv',sep='\t',index=False)

In [176]:
process_external_data['ner_e1'] = process_external_data.iloc[:,2].apply(lambda x: ner(x))
process_external_data['ner_e2'] = process_external_data.iloc[:,5].apply(lambda x: ner(x))

In [187]:
process_external_data.to_csv('../input/data/external_data/ner_external_data.tsv',sep='\t',index=False)

In [177]:
new_test = test.copy()
new_test['ner_e1'] = new_test.iloc[:,2].apply(lambda x: ner(x))
new_test['ner_e2'] = new_test.iloc[:,5].apply(lambda x: ner(x))

In [188]:
new_test.to_csv('../input/data/test/ner_test.tsv',sep='\t',index=False)

In [178]:
# new_train은 원래 train의 형식으로 맞춰서 저장하자.

# new_train_drop.to_csv('../input/data/train/new_train.tsv',sep='\t',index=False)

In [179]:
# 서장훈 index가 잘못적혀있음.
new_train_drop.iloc[9103,[3,4]] = [12,14]

In [180]:
# input_series = new_train_drop.iloc[4]
def add_ner_token(myseries):
  original_sentence = myseries.iloc[1]
  ent1, ent1_start, ent1_end = myseries.iloc[2], int(myseries.iloc[3]), int(myseries.iloc[4])
  ent2, ent2_start, ent2_end = myseries.iloc[5], int(myseries.iloc[6]), int(myseries.iloc[7])

  ner_tag1 = ' '.join([x[1] for x in myseries.loc['ner_e1'] if x[1]!='O'])
  ner_tag2 = ' '.join([x[1] for x in myseries.loc['ner_e2'] if x[1]!='O'])


  # result_ner_tag1 = ' @' + ' * ' + ner_tag1.lower() + ' * ' +  + ' @'
  # result_ner_tag2 = ' #' + ' ^ ' + ner_tag2.lower() + ' ^ ' + input_series.iloc[5] + ' #'

  add_ner_sentence = original_sentence[:ent1_start] + ' @' + ' * ' + ner_tag1.lower() + ' * ' + original_sentence[ent1_start:ent1_end+1] + ' @' + \
                    original_sentence[ent1_end+1:ent2_start] + ' #' + ' ^ ' + ner_tag2.lower() + ' ^ ' + original_sentence[ent2_start:ent2_end+1] + ' #' + original_sentence[ent2_end+1:]

  return add_ner_sentence
# print(add_ner_sentence)
# print(result_ner_tag2)


In [183]:
final_train_ner = new_train_drop.copy()
final_train_ner.iloc[:,1] = final_train_ner.apply(lambda x: add_ner_token(x), axis=1)
final_train_ner = final_train_ner.drop(['token_len','ner_e1','ner_e2'],axis=1)
final_train_ner

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 @ * organizatio...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,선거에서 @ * organization * 민주당 @은 해산 전 의석인 230석에...,민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,@ * organization * 유럽 축구 연맹 @( # ^ organizati...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,용병 공격수 챠디의 부진과 시즌 초 활약한 @ * person * 강수일 @ # ...,강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,@ * location * 람캄행 @ 왕은 1237년에서 1247년 사이 수코타이...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
9103,wikitree-55169-1-15-17-23-27,10일 매체는 방송인 @ * person * 서장훈 @이 모교 연세대 # ^ or...,서장훈,12,14,연세대학교,23,27,인물:학교
9104,wikipedia-29295-6-74-76-60-64,"이때 리천희는 주포 양희승이 아킬레스건 상처를 입었을 때, 주전 선수들이 고려대학교...",이규섭,74,76,고려대학교,60,64,인물:학교
9105,wikipedia-9102-0-0-5-49-53,@ * person * 다쓰노 긴고 @(1854년 10월 13일~1919년 3월 ...,다쓰노 긴고,0,5,고부대학교,49,53,인물:학교
9106,wikipedia-14613-6-76-81-22-27,"1949년~1950년에는 교환학생으로 파리 대학교에 머물렀고, 여기서 장 룰레 안리...",자크 디미에,76,81,파리 대학교,22,27,인물:학교


In [184]:
final_external_ner = process_external_data.copy()
final_external_ner.iloc[:,1] = final_external_ner.apply(lambda x: add_ner_token(x), axis=1)
final_external_ner = final_external_ner.drop(['token_len','ner_e1','ner_e2'],axis=1)
final_external_ner

,0,1,2,3,4,5,6,7,8
0,from dev_csv 5135,2014년 @ * organization * MBC @주말특별기획 《 # ^ a...,MBC,6,9,마마 (드라마),18,26,단체:제작
1,from train_csv 283731,2014 년 @ * organization * tvN @월화 드라마 《 # ^...,tvN,7,10,라이어 게임,20,26,단체:제작
2,from train_csv 350859,《 막돼먹은 영애씨 》 는 @ * organization * tvN @ # ^ ...,tvN,15,18,막돼먹은 영애씨,2,10,단체:제작
3,from train_csv 242038,정재형 이효리의 유&아이 ( 2012 . 02 . 26 ~ 2012 . 10 . 1...,SBS TV,51,57,정재형 이효리의 유&아이,1,14,단체:제작
4,from train_csv 268241,KBS 뉴스라인 ( K @ * organization artifact * BS 제1...,KBS 제1TV,12,20,KBS 뉴스라인,1,9,단체:제작
...,...,...,...,...,...,...,...,...,...
1201,from dev_csv 8366,이에 활동이 멈출 위기에 놓였으나 휴렛 팩커드의 창업자인 윌리엄 휴렛과 데이브 팩커...,폴 앨런,65,69,마이크로소프트,49,56,단체:창립자
1202,from train_csv 975423,"1832년, 제 @ * person * 임스 매터슨과 @청나라 광저우(沙面島)에서...",제임스 매터슨,8,15,자딘 매터슨,33,39,단체:창립자
1203,from dev_csv 8346,"유한양행과 라초이를 설립한 기업가 유일한의 동생이자, 유유제약을 설립한 @ * ...",유특한,40,43,유유제약,30,34,단체:창립자
1204,from dev_csv 8321,현재 그는 @ * person * T.I.가 @ 설립한 레이블인 # ^ ^ 그...,T.I.,6,10,그랜드 허슬 레코드,21,31,단체:창립자


In [185]:
final_test_ner = new_test.copy()
final_test_ner.iloc[:,1] = final_test_ner.apply(lambda x: add_ner_token(x), axis=1)
final_test_ner = final_test_ner.drop(['token_len','ner_e1','ner_e2'],axis=1)
final_test_ner

,0,1,2,3,4,5,6,7,8
0,wikipedia-15133-19-0-2-45-46,"@ * person * 이은주 @의 첫 영화로, 그녀가 강수연(정화 역)의 동생 ...",이은주,0,2,감독,45,46,blind
1,wikitree-10241-0-0-4-9-11,@ * organization * 경성대학교 @(총장 # ^ ^ 송수건 #) ...,경성대학교,0,4,송수건,9,11,blind
2,wikipedia-24909-38-83-89-45-48,하더라도 만일 그녀의 아들에게 계승시킬 수 있다면 영국 왕 에드워드 2세의 왕비 이...,에드워드 3세,83,89,이사벨라,45,48,blind
3,wikitree-29775-2-38-43-17-18,정성으로 담근 700포기의 김장김치는 발굴된 저소득 가정 55세대에 @ * org...,새마을부녀회,38,43,김치,17,18,blind
4,wikipedia-20874-21-8-9-30-32,"노트북은 이들 @ * organization * 정부 @에 판매될 것이며 ""하나의...",정부,8,9,한 명,30,32,blind
...,...,...,...,...,...,...,...,...,...
995,wikipedia-5911-11-14-18-21-23,"1940년 7월 10일, @ * organization country * 나치 독...",나치 독일,14,18,프랑스,21,23,blind
996,wikipedia-17379-20-200-205-179-181,(하지만 Vasco는 '스핏파이어(spit fire)(spit fire)'란 프로젝...,DJ DOC,200,205,박정환,179,181,blind
997,wikitree-59796-5-32-35-37-39,또한 (사)한국여성농업인 광양시 연합회(도미화 회장)는 ㈜ @ * organizat...,광양교통,32,35,이순심,37,39,blind
998,wikipedia-1829-41-9-10-0-1,마코가 항의하자 @ * animal * 벵어 @ # ^ person ^ 마코 #가...,벵어,9,10,마코,0,1,blind


In [18]:
import pandas as pd
final_train_ner = pd.read_csv("../input/data/train/final_train_ner.tsv",sep='\t')
final_external_ner = pd.read_csv("../input/data/external_data/final_external_ner.tsv",sep='\t')
final_test_ner = pd.read_csv("../input/data/test/final_test_ner.tsv",sep='\t')
final_train_ner

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 @ * organizatio...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,선거에서 @ * organization * 민주당 @은 해산 전 의석인 230석에...,민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,@ * organization * 유럽 축구 연맹 @( # ^ organizati...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,용병 공격수 챠디의 부진과 시즌 초 활약한 @ * person * 강수일 @ # ...,강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,@ * location * 람캄행 @ 왕은 1237년에서 1247년 사이 수코타이...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
9103,wikitree-55169-1-15-17-23-27,10일 매체는 방송인 @ * person * 서장훈 @이 모교 연세대 # ^ or...,서장훈,12,14,연세대학교,23,27,인물:학교
9104,wikipedia-29295-6-74-76-60-64,"이때 리천희는 주포 양희승이 아킬레스건 상처를 입었을 때, 주전 선수들이 고려대학교...",이규섭,74,76,고려대학교,60,64,인물:학교
9105,wikipedia-9102-0-0-5-49-53,@ * person * 다쓰노 긴고 @(1854년 10월 13일~1919년 3월 ...,다쓰노 긴고,0,5,고부대학교,49,53,인물:학교
9106,wikipedia-14613-6-76-81-22-27,"1949년~1950년에는 교환학생으로 파리 대학교에 머물렀고, 여기서 장 룰레 안리...",자크 디미에,76,81,파리 대학교,22,27,인물:학교


In [10]:
final_train_ner

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 @ * organizatio...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,선거에서 @ * organization * 민주당 @은 해산 전 의석인 230석에...,민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,@ * organization * 유럽 축구 연맹 @( # ^ organizati...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,용병 공격수 챠디의 부진과 시즌 초 활약한 @ * person * 강수일 @ # ...,강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,@ * location * 람캄행 @ 왕은 1237년에서 1247년 사이 수코타이...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
9103,wikitree-55169-1-15-17-23-27,10일 매체는 방송인 @ * person * 서장훈 @이 모교 연세대 # ^ or...,서장훈,12,14,연세대학교,23,27,인물:학교
9104,wikipedia-29295-6-74-76-60-64,"이때 리천희는 주포 양희승이 아킬레스건 상처를 입었을 때, 주전 선수들이 고려대학교...",이규섭,74,76,고려대학교,60,64,인물:학교
9105,wikipedia-9102-0-0-5-49-53,@ * person * 다쓰노 긴고 @(1854년 10월 13일~1919년 3월 ...,다쓰노 긴고,0,5,고부대학교,49,53,인물:학교
9106,wikipedia-14613-6-76-81-22-27,"1949년~1950년에는 교환학생으로 파리 대학교에 머물렀고, 여기서 장 룰레 안리...",자크 디미에,76,81,파리 대학교,22,27,인물:학교


In [7]:
sum(final_external_ner.iloc[:,8].apply(lambda x: '8' in x))


0

In [186]:
final_train_ner.to_csv('../input/data/train/final_train_ner.tsv',sep='\t',index=False)
final_external_ner.to_csv('../input/data/external_data/final_external_ner.tsv',sep='\t',index=False)
final_test_ner.to_csv('../input/data/test/final_test_ner.tsv',sep='\t',index=False)


In [522]:
'∧'

'∧'